In [1]:
import shutil
import sys


shutil.copytree('/kaggle/input/packages/fvcore', 'fvcore')
shutil.copytree('/kaggle/input/packages/iopath', 'iopath')
shutil.copytree('/kaggle/input/packages/pytorchvideo', 'pytorchvideo')
shutil.copytree('/kaggle/input/packages/NFL-Player-Contact-Detection', 'NFL-Player-Contact-Detection')
shutil.copytree('/kaggle/input/models', 'models')


sys.path.append('/kaggle/working/NFL-Player-Contact-Detection')
sys.path.append('/kaggle/working/models')
sys.path.append('/kaggle/working/pytorchvideo')
sys.path.append('/kaggle/working/fvcore')
sys.path.append('/kaggle/working/iopath')


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import io

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms 
from torch.nn.utils.rnn import pad_sequence

import os
import cv2
from sklearn.metrics import matthews_corrcoef, roc_auc_score, classification_report

from trainer import Trainer
from frame_extraction import FrameExctracor, extract_parallel
from models import FilterGRU, TransformerTracker 


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
if device == 'cuda:0':
    print('GPU: ', torch.cuda.get_device_name())


GPU:  Tesla P100-PCIE-16GB


## data

In [3]:
import datasets
from torchvision import transforms
import torch
import io
import os
import cv2

def create_image_dataset(source_dir, file_names = [], transform = None):
    
    if not file_names:
        file_names = os.listdir(source_dir)
        
    i_paths = [f'{source_dir}/{p}' for p in file_names]
    
    image_data = datasets.Dataset\
        .from_dict({'file_path': i_paths})\
        .save_to_disk('image_data')
    
    image_data = datasets.Dataset.load_from_disk('image_data')

    def load_imgs(examples):
        imgs = []
        for ex in examples['file_path']:
            img = cv2.imread(ex)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img= torch.from_numpy(img).permute(2,0,1)
            else:
                img=torch.zeros(3,720,1280)
            if transform is not None:
                img = transform(img)
                
            b = io.BytesIO()
            torch.save(img, b)
            b.seek(0)
            imgs.append(b.read())

        return {'image': imgs}
    
    image_data = image_data.map(load_imgs, batched=True, batch_size=600)

    path2index = {p.split("/")[-1].replace(".jpg", ""):i for i, p in enumerate(image_data['file_path'])}
    index2path, _ = zip(*sorted(path2index.items(), key=lambda x: x[1]))

    def b2tensor(examples):
        examples['image'] = [torch.load(io.BytesIO(image)) for image in examples['image']]
        return examples

    image_data.set_transform(b2tensor)
    
    return image_data, path2index, index2path

In [4]:

keys = [ 'game_play',  'nfl_player_id_1', 'nfl_player_id_2', 'jersey_number_p1', 'position_p1', 'jersey_number_p2', 'position_p2', ]
features = ['contact_id', 'frame', 'contact', 'step', 
            'acceleration_p1',  'acceleration_p2', 'speed_p2', 'speed_p1', 'sa_p1', 'sa_p2','dist', 'is_one_team', 'is_ground_contact']

def agg_chanks_fn(df, chank_size):
    res = {_: [] for _ in keys + features}

    for d in np.array_split(df, len(df)//chank_size + 1):
        if  not d.empty:
            for k in keys:
                res[k].append(d[k].iloc[0])
            for f in features:
                res[f].append(d[f].tolist())
                
    return pd.DataFrame(res)


def create_tracking_features(labels, tracking, chank_size = None,
                             dist_threshold = 4, speed_threshold= 4, 
                             keys=keys, features= features):
    
    p_df = labels.query('nfl_player_id_2 != "G"')\
            .merge(tracking.rename({'nfl_player_id': 'nfl_player_id_1'}, axis=1))\
            .merge(tracking.rename({'nfl_player_id': 'nfl_player_id_2'}, axis=1), 
                   on = ['game_play', 'game_key', 'play_id', 'datetime', 'step', 'frame', 'nfl_player_id_2'], 
                   suffixes=['_p1', '_p2'])


    p_df['dist'] = np.sqrt((p_df.y_position_p1-p_df.y_position_p2)**2 + (p_df.x_position_p1-p_df.x_position_p2)**2)
    p_df['is_one_team'] = (p_df.team_p1==p_df.team_p2).astype(int)
    p_df = p_df.query(f"dist<{dist_threshold}")
    p_df['is_ground_contact'] = 0
    
    
    
    
    # ground contacts
    g_df=labels.query('nfl_player_id_2 == "G"')\
        .merge(tracking.rename({'nfl_player_id': 'nfl_player_id_1'}, axis=1))
        
    g_df = g_df.query(f"speed<{speed_threshold}")
    
    g_df.columns = p_df.columns[:len(g_df.columns)]
    g_df.loc[g_df.contact == 1, 'contact'] = 2
    
    g_df['is_ground_contact'] = 1
    
    
    # put together
    df = pd.concat([p_df,g_df])\
        .fillna(0)\
        .sort_values(keys + ['step'])
    


    if chank_size is not None:
        df = df.groupby(keys, group_keys=False).apply(agg_chanks_fn, chank_size)     
    else:
        df = df.groupby(keys, as_index=False).agg(list)[keys+features]
     
    return df


def crop_players(img, p1_id, p2_id, game_play, players, zone):
    
    mask = torch.zeros(img.shape)
    show_list = players[players.nfl_player_id.isin([p1_id, p2_id]) & (players.view == zone)]
    hide_list = players[~players.nfl_player_id.isin([p1_id, p2_id]) & (players.view == zone)]
    
    if show_list.empty:
        return mask

    xmin = 100000
    xmax = 0
    ymin = 100000
    ymax = 0

    for i, p in show_list.iterrows():
        x1 = int(p.left - 2*p.width)
        x2 = int(p.left + 5*p.width)
        y1 = int(p.top)
        y2 = int(p.top + 8*p.height) 
        mask[:, y1:y2, x1:x2] = 1

        xmin = min(x1, xmin)
        xmax = max(x2, xmax)
        ymin = min(y1, ymin)
        ymax = max(y2, ymax)


    """for i, p in hide_list.iterrows():
        x1 = int(p.left - 1*p.width)
        x2 = int(p.left + 2*p.width)
        y1 = int(p.top)
        y2 = int(p.top + 8*p.height) 
        mask[:, y1:y2, x1:x2] = 0
    """
    
    xmin = max(xmin,0)
    xmax = min(xmax, img.shape[2])
    ymin = max(ymin,0)
    ymax = min(ymax, img.shape[1])
    size = max((ymax-ymin), (xmax-xmin) )
    
    c, h, w = img.shape
    res = torch.zeros((c, h + size, w + size))
    res[:,:h, :w] = torch.masked_fill(img, mask==0, 0)
    
    
    return res[:, ymin:ymin+size, xmin:xmin+size, ]

    
class PlayerContactDataset(torch.utils.data.Dataset):
    def __init__(self, tracking_features , helmets = None, transform = None, 
                 keys=keys, features= features, image_data=None, pad_value=-100):
        super().__init__()
        self.features= features
        self.transform = transform
        self.df = tracking_features
        self.image_data = image_data
        self.load_image = image_data is not None
        self.helmets = helmets
        self.pad_value = pad_value
        
        
    def __getitem__(self, idx):
        
        row = self.df.iloc[idx]
        X = torch.tensor(row[self.features].tolist()[-9:]).T
        
        res = {
            'contact_id': row.contact_id,
            'X': X,
            'label': torch.tensor(row.contact),
        }
        
        if self.load_image:
            imgs = torch.zeros((len(row.frame)*2,3, 224, 224))

            for f_id, frame in enumerate(row.frame):
                try:
                    players = self.helmets.loc[(row.game_play, frame)]
                except KeyError:
                    players = None
                    
                if players is not None:  
                    for z_id, zone in enumerate(['Endzone', 'Sideline']):

                        idx = path2index.get(f'{row.game_play}_{zone}_{frame}')

                        if idx:
                            img = self.image_data[idx]['image']
                            img = crop_players(img,row.nfl_player_id_1, row.nfl_player_id_2, row.game_play, players, zone)
                            
                            imgs[f_id*2+z_id]=transforms.functional.resize(img,[224,224])
                        
            res['imgs'] = imgs.div(255)
            
            if self.transform is not None:
                res['imgs'] = self.transform(res['imgs'])
                    

        return res
    
    def __len__(self):
        return len(self.df)
    
    def _pad(self, seq, key):
        if torch.is_tensor(seq[0]):
            if key == 'imgs':
                pad_val = 0
            elif key =='label':
                pad_val = -100
            else:
                pad_val = self.pad_value
                
            res = pad_sequence(seq,batch_first=True, padding_value=pad_val)
        else:
            max_lenght = max(len(_) for _ in seq)
            res = [_ + (max_lenght-len(_))*[self.pad_value] for _ in seq]
        return res
    
    def collate_fn(self, batch):
        values = zip(*[b.values() for b in batch])
        keys = batch[0].keys()
        return {k: self._pad(v, k) for k, v in zip(keys, values)}
    

In [5]:
#create_tracking_features
tracking = pd.read_csv('/kaggle/input/nfl-player-contact-detection/test_player_tracking.csv').astype({'nfl_player_id': str})
tracking['frame'] = (59.94*0.1*tracking['step'] + 5*59.94).astype(int)
positions = tracking.position.unique().tolist()
positions = {p: i for i, p in enumerate(positions)}
tracking['position'] = tracking['position'].apply(lambda x: positions[x])


labels = pd.read_csv('/kaggle/input/nfl-player-contact-detection/sample_submission.csv')

def split_fn(x):
    x = x.split('_')
    return [f'{x[0]}_{x[1]}'] + x[2:]


labels = pd.concat([labels, pd.DataFrame(
                    labels.contact_id.apply(split_fn).tolist(),
                    columns =['game_play', 'step', 'nfl_player_id_1', 'nfl_player_id_2']
                ).astype({'step': int, 'nfl_player_id_1': str, 'nfl_player_id_2': str})],
          axis=1)


tf= create_tracking_features(labels, tracking, chank_size=20)


# filtering 
dataset = PlayerContactDataset(tf)

filter_model=  FilterGRU().from_pretrained('/kaggle/input/models/filter').to(device)

def dummy_loss(*args, **kwargs):
    return torch.tensor(0.)

def get_probas(logits, label, **kwargs):
    
    y_pred = (logits.softmax(-1)[:,1:].sum(-1)>0.2).type(torch.uint8)
    y_true = (label.max(-1)[0]>0).type(torch.uint8)

    return y_true, y_pred

trainer = Trainer(
    dataset,
    dataset,
    model = filter_model,            
    batch_size = 1000,
    num_epoch = 1,
    loss_fn = dummy_loss,
    post_procc_fn = get_probas
)

result = trainer.evaluation()[1]                         
dataset.df = dataset.df[np.array(result['preds'])==1]
tf =dataset.df

frames4extract = tf[['game_play', 'frame']].groupby('game_play', as_index = False).agg(sum)
frames4extract.loc[:,'frame'] = frames4extract.frame.apply(lambda x: set(x))


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [6]:
#image data
extract_parallel(frames4extract, '/kaggle/input/nfl-player-contact-detection/test', 'frames')
image_data, path2index, index2path = create_image_dataset('frames', transform = transforms.Resize(256))

#load helmets data
h,w = image_data[0]['image'].shape[1:]

helmets = pd.read_csv('/kaggle/input/nfl-player-contact-detection/test_baseline_helmets.csv').astype({'nfl_player_id': str})
helmets['key'] = helmets.apply(lambda x: f"{x['game_play']}_{x['view']}_{x['frame']}", axis=1)

helmets.loc[:, ['left', 'width']] = (helmets.loc[:, ['left', 'width']]*w/1280).astype(int)
helmets.loc[:, ['top', 'height']] = (helmets.loc[:, ['top', 'height']]*h/720).astype(int)

helmets = pd.DataFrame({'key': path2index.keys()}).merge(helmets)
helmets = helmets.set_index(['game_play', 'frame']).sort_index()

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:17<00:00, 17.87s/it]


  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
std=np.array([0.229, 0.224, 0.225])
mean=np.array([0.485, 0.456, 0.406])

transform = nn.Sequential(
    transforms.Normalize(std=std, mean=mean)
)

train_size = int(len(tf)*0.9)

dataset = PlayerContactDataset(
    tf, pad_value=0, transform = transform, 
    image_data=image_data, helmets = helmets
)

## model

In [8]:
import torch
from torch import nn
import os
import math
from models.pretrained_model import PreTrainedModel

a= [1]

class X3D_GRU(nn.Module):
    def __init__(self, 
            x3d_kwargs = {
                'repo_or_dir' :'pytorchvideo',
                'model': 'x3d_s',
                'source': 'local', 
                'pretrained': False
            }
        ):
        super().__init__()
        
        self.x3d = torch.hub.load(**x3d_kwargs)
        self.x3d.blocks[5]= nn.Identity()
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        
        num_features = list(self.x3d.blocks[4].res_blocks.modules())[-2].num_features*2
        
        self.gru = nn.GRU(input_size =num_features, hidden_size=num_features, num_layers=3, 
                          bidirectional=True, batch_first=True)
        self.fc = nn.Linear(num_features*2,3)
        
          
    def forward(self, imgs,  **kwargs):
        
        f = self.x3d(imgs.permute(0,2,1,3,4))
        
        f = self.pool(f)\
            .squeeze(-1).squeeze(-1)\
            .permute(0,2,1)\
            .reshape(f.shape[0], f.shape[2]//2, -1 )
        
        X, h = self.gru(f)
        return self.fc(X)


class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)
                        * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


class TransformerTracker(PreTrainedModel):
    def __init__(self, ):
        super().__init__()
        self.upsample = nn.Linear(9,256)
        self.positional_encoding = PositionalEncoding(256, dropout=0.1)

        encoder_layer = nn.TransformerEncoderLayer(d_model=256, nhead=4, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=4)
        self.fc = nn.Linear(256,3)
        self.x3d = X3D_GRU()
        self.x3d.fc = nn.Linear(768,256)

        
    def forward(self, X, imgs,  **kwargs):
            
        X = self.positional_encoding(self.upsample(X))
        X = self.transformer_encoder(X)
        try:
            imgs = self.x3d(imgs)
        except:
            a[0] = imgs.cpu().numpy()
        
        return self.fc(X+imgs)




## submission

In [9]:
model = TransformerTracker().from_pretrained('/kaggle/input/models/contact_detector').to(device)

def post_procc(logits, label, **kawargs):
    ids = label.flatten() != -100
    y_pred = (1-logits.softmax(-1)[:,:,0]).flatten()[ids]
    y_true = label.flatten()[ids]
    return y_true, (y_pred>0.44).to(dtype=torch.int32) 
  
trainer = Trainer(
    dataset,
    dataset,
    model = model,            
    batch_size = 4,
    num_epoch = 1,
    loss_fn = dummy_loss,
    post_procc_fn = post_procc
)

result= trainer.evaluation()[1]

df = pd.DataFrame({'contact_id': dataset.df.contact_id.agg(sum), 'preds': result['preds']})
submission = pd.read_csv('/kaggle/input/nfl-player-contact-detection/sample_submission.csv')
submission.contact = submission.merge(df, how='left')\
    .preds.fillna(0)\
    .astype(int)
submission.loc[submission.contact==2, 'contact'] = 1
submission.to_csv('submission.csv',  index=False)

In [10]:
for p in os.listdir('.'):
    if p != 'submission.csv' and os.path.isdir(p):
        shutil.rmtree(p)